In [1]:
# Import the modules
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
#Read new csv file in
file_path = Path("Resources/realtor-data.zip.csv")
df = pd.read_csv(file_path)

# Display sample data
df.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,NaN,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,NaN,NaN,65000.0


In [3]:
df.drop(columns=['status','zip_code','prev_sold_date'], inplace=True)

In [4]:
df.head()


,bed,bath,acre_lot,city,state,house_size,price
0,3.0,2.0,0.12,Adjuntas,Puerto Rico,920.0,105000.0
1,4.0,2.0,0.08,Adjuntas,Puerto Rico,1527.0,80000.0
2,2.0,1.0,0.15,Juana Diaz,Puerto Rico,748.0,67000.0
3,4.0,2.0,0.10,Ponce,Puerto Rico,1800.0,145000.0
4,6.0,2.0,0.05,Mayaguez,Puerto Rico,NaN,65000.0


In [5]:
df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680545 entries, 0 to 1401065
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   bed         680545 non-null  float64
 1   bath        680545 non-null  float64
 2   acre_lot    680545 non-null  float64
 3   city        680545 non-null  object 
 4   state       680545 non-null  object 
 5   house_size  680545 non-null  float64
 6   price       680545 non-null  float64
dtypes: float64(5), object(2)
memory usage: 41.5+ MB


In [7]:
# # Define minimum and maximum thresholds
# # min_threshold = 10000  # Adjust as needed
# max_threshold = 500000 # Adjust as needed

# # Filter rows based on the condition
# filtered_df = df[(df['price'] >= min_threshold) & (df['price'] <= max_threshold)]

# # Display the filtered DataFrame
# filtered_df


In [8]:
# Define the threshold
threshold_value = 500000  # Adjust as needed

# Filter rows based on the condition
filtered_df = df[df['price'] <= threshold_value]

# Display the filtered DataFrame
filtered_df


,bed,bath,acre_lot,city,state,house_size,price
0,3.0,2.0,0.12,Adjuntas,Puerto Rico,920.0,105000.0
1,4.0,2.0,0.08,Adjuntas,Puerto Rico,1527.0,80000.0
2,2.0,1.0,0.15,Juana Diaz,Puerto Rico,748.0,67000.0
3,4.0,2.0,0.10,Ponce,Puerto Rico,1800.0,145000.0
5,4.0,3.0,0.46,San Sebastian,Puerto Rico,2520.0,179000.0
...,...,...,...,...,...,...,...
1401060,3.0,1.0,0.10,Dunkirk,New York,1002.0,29900.0
1401062,4.0,2.0,0.36,Silver Creek,New York,2026.0,187900.0
1401063,10.0,4.0,0.43,Brocton,New York,4802.0,120000.0
1401064,2.0,2.0,0.14,Dunkirk,New York,1568.0,92000.0


In [9]:
filtered_df['price'].describe()

count    396840.000000
mean     280545.184596
std      121567.377733
min           1.000000
25%      182000.000000
50%      275000.000000
75%      379000.000000
max      500000.000000
Name: price, dtype: float64

In [10]:
# Define the number of bins
num_bins = 3

# Binning the prices
bins = pd.cut(filtered_df['price'], bins=num_bins, labels=False, include_lowest=True)

# Sort bins in ascending order
sorted_bins = bins.rank(method='dense').astype(int) - 1

# Create a new DataFrame with the sorted bin information appended
df_with_bins = filtered_df.copy()
df_with_bins['price_bin'] = sorted_bins

# Display the new DataFrame with the bin information
df_with_bins


,bed,bath,acre_lot,city,state,house_size,price,price_bin
0,3.0,2.0,0.12,Adjuntas,Puerto Rico,920.0,105000.0,0
1,4.0,2.0,0.08,Adjuntas,Puerto Rico,1527.0,80000.0,0
2,2.0,1.0,0.15,Juana Diaz,Puerto Rico,748.0,67000.0,0
3,4.0,2.0,0.10,Ponce,Puerto Rico,1800.0,145000.0,0
5,4.0,3.0,0.46,San Sebastian,Puerto Rico,2520.0,179000.0,1
...,...,...,...,...,...,...,...,...
1401060,3.0,1.0,0.10,Dunkirk,New York,1002.0,29900.0,0
1401062,4.0,2.0,0.36,Silver Creek,New York,2026.0,187900.0,1
1401063,10.0,4.0,0.43,Brocton,New York,4802.0,120000.0,0
1401064,2.0,2.0,0.14,Dunkirk,New York,1568.0,92000.0,0


In [11]:
# Check unique values of the 'price_bin' column
print(df_with_bins['price_bin'].unique())


[0 1 2]


In [12]:
# Compute bin edges
bins = pd.cut(df_with_bins['price'], bins=num_bins, include_lowest=True).unique()

# Display the bin edges
print("Bin Edges:")
print(bins)


Bin Edges:
[(-499.0, 166667.333], (166667.333, 333333.667], (333333.667, 500000.0]]
Categories (3, interval[float64, right]): [(-499.0, 166667.333] < (166667.333, 333333.667] < (333333.667, 500000.0]]


Encode Categorical Variables

In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Select categorical columns
cat_columns = ['city','state']

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(drop='first', sparse=False), cat_columns)],
    remainder='passthrough'
)

data_encoded = ct.fit_transform(df_with_bins)

# Now 'data_encoded' contains numerical representations of categorical variables

encoded_df = pd.DataFrame(data_encoded, columns=ct.get_feature_names_out())

# Concatenate encoded DataFrame with the original DataFrame
merged_data = pd.concat([filtered_df.drop(columns=cat_columns), encoded_df], axis=1)



/Users/tiffanyphipps/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
encoded_df

,one_hot_encoder__city_Aberdeen,one_hot_encoder__city_Abington,one_hot_encoder__city_Absecon,one_hot_encoder__city_Accord,one_hot_encoder__city_Acra,one_hot_encoder__city_Acton,one_hot_encoder__city_Acushnet,one_hot_encoder__city_Acworth,one_hot_encoder__city_Adams,one_hot_encoder__city_Adams Center,...,one_hot_encoder__state_Rhode Island,one_hot_encoder__state_Vermont,one_hot_encoder__state_Virgin Islands,one_hot_encoder__state_West Virginia,remainder__bed,remainder__bath,remainder__acre_lot,remainder__house_size,remainder__price,remainder__price_bin
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.12,920.0,105000.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.08,1527.0,80000.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.15,748.0,67000.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.10,1800.0,145000.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,3.0,0.46,2520.0,179000.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,0.10,1002.0,29900.0,0.0
396836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.36,2026.0,187900.0,1.0
396837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,4.0,0.43,4802.0,120000.0,0.0
396838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.14,1568.0,92000.0,0.0


In [15]:
merged_data

,bed,bath,acre_lot,house_size,price,one_hot_encoder__city_Aberdeen,one_hot_encoder__city_Abington,one_hot_encoder__city_Absecon,one_hot_encoder__city_Accord,one_hot_encoder__city_Acra,...,one_hot_encoder__state_Rhode Island,one_hot_encoder__state_Vermont,one_hot_encoder__state_Virgin Islands,one_hot_encoder__state_West Virginia,remainder__bed,remainder__bath,remainder__acre_lot,remainder__house_size,remainder__price,remainder__price_bin
0,3.0,2.0,0.12,920.0,105000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.12,920.0,105000.0,0.0
1,4.0,2.0,0.08,1527.0,80000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.08,1527.0,80000.0,0.0
2,2.0,1.0,0.15,748.0,67000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.15,748.0,67000.0,0.0
3,4.0,2.0,0.10,1800.0,145000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.10,1800.0,145000.0,0.0
5,4.0,3.0,0.46,2520.0,179000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,0.20,2040.0,50000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396826,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.25,696.0,139000.0,0.0
396827,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,5.50,1144.0,89900.0,0.0
396828,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,2.00,1723.0,249900.0,1.0
396832,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,4.60,1470.0,145000.0,0.0


Scale Numerical Features

In [16]:
merged_data = merged_data.dropna()

In [17]:
# Select numerical columns
num_columns = ['bed', 'bath', 'acre_lot', 'house_size']

# Apply standardization (z-score normalization)
scaler = StandardScaler()
df_with_bins[num_columns] = scaler.fit_transform(df_with_bins[num_columns])

# Concatenate 'merged_data' (encoded categorical variables) with 'df' (scaled numerical features)
# Concatenate 'merged_data' (encoded categorical variables) with 'df' (scaled numerical features)
final_data = pd.concat([df_with_bins.drop(columns=num_columns + cat_columns + ["price"]), merged_data], axis=1)

# Now 'final_data' contains the merged DataFrame with duplicate columns removed



In [18]:
final_data

,price_bin,bed,bath,acre_lot,house_size,price,one_hot_encoder__city_Aberdeen,one_hot_encoder__city_Abington,one_hot_encoder__city_Absecon,one_hot_encoder__city_Accord,...,one_hot_encoder__state_Rhode Island,one_hot_encoder__state_Vermont,one_hot_encoder__state_Virgin Islands,one_hot_encoder__state_West Virginia,remainder__bed,remainder__bath,remainder__acre_lot,remainder__house_size,remainder__price,remainder__price_bin
0,0,3.0,2.0,0.12,920.0,105000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.12,920.0,105000.0,0.0
1,0,4.0,2.0,0.08,1527.0,80000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.08,1527.0,80000.0,0.0
2,0,2.0,1.0,0.15,748.0,67000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.15,748.0,67000.0,0.0
3,0,4.0,2.0,0.10,1800.0,145000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.10,1800.0,145000.0,0.0
5,1,4.0,3.0,0.46,2520.0,179000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,0.20,2040.0,50000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401060,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401062,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401063,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401064,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
final_data = final_data.dropna()

In [20]:
final_data

,price_bin,bed,bath,acre_lot,house_size,price,one_hot_encoder__city_Aberdeen,one_hot_encoder__city_Abington,one_hot_encoder__city_Absecon,one_hot_encoder__city_Accord,...,one_hot_encoder__state_Rhode Island,one_hot_encoder__state_Vermont,one_hot_encoder__state_Virgin Islands,one_hot_encoder__state_West Virginia,remainder__bed,remainder__bath,remainder__acre_lot,remainder__house_size,remainder__price,remainder__price_bin
0,0,3.0,2.0,0.12,920.0,105000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.12,920.0,105000.0,0.0
1,0,4.0,2.0,0.08,1527.0,80000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.08,1527.0,80000.0,0.0
2,0,2.0,1.0,0.15,748.0,67000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.15,748.0,67000.0,0.0
3,0,4.0,2.0,0.10,1800.0,145000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.10,1800.0,145000.0,0.0
5,1,4.0,3.0,0.46,2520.0,179000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,0.20,2040.0,50000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396834,2,4.0,2.0,0.92,1982.0,350000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,2.0,0.55,4260.0,95000.0,0.0
396836,0,2.0,1.0,0.04,672.0,34900.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.36,2026.0,187900.0,1.0
396837,1,4.0,2.0,0.16,1248.0,249900.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,4.0,0.43,4802.0,120000.0,0.0
396838,0,6.0,2.0,0.12,2478.0,165000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.14,1568.0,92000.0,0.0


In [21]:
X = final_data.copy()
X.drop('price_bin', axis = 1, inplace=True)
X.head()

,bed,bath,acre_lot,house_size,price,one_hot_encoder__city_Aberdeen,one_hot_encoder__city_Abington,one_hot_encoder__city_Absecon,one_hot_encoder__city_Accord,one_hot_encoder__city_Acra,...,one_hot_encoder__state_Rhode Island,one_hot_encoder__state_Vermont,one_hot_encoder__state_Virgin Islands,one_hot_encoder__state_West Virginia,remainder__bed,remainder__bath,remainder__acre_lot,remainder__house_size,remainder__price,remainder__price_bin
0,3.0,2.0,0.12,920.0,105000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.12,920.0,105000.0,0.0
1,4.0,2.0,0.08,1527.0,80000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.08,1527.0,80000.0,0.0
2,2.0,1.0,0.15,748.0,67000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0.15,748.0,67000.0,0.0
3,4.0,2.0,0.10,1800.0,145000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,2.0,0.10,1800.0,145000.0,0.0
5,4.0,3.0,0.46,2520.0,179000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,0.20,2040.0,50000.0,0.0


In [22]:
# Define target vector
y = final_data["price_bin"].ravel()
y[:5]


array([0, 0, 0, 0, 1])

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [24]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [25]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [26]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [27]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Fitting the Random Forest Model

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=300, random_state=78)

In [30]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

Making Predictions Using the Random Forest Model

In [31]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

Model Evaluation

In [32]:
import numpy as np

# Assuming 'y' is your target variable
num_classes = np.unique(y).size

print("Number of classes:", num_classes)


Number of classes: 3


In [33]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2"], columns=["Predicted 0", "Predicted 1", "Predicted 2"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [34]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2
Actual 0,5301,82,0
Actual 1,13,13724,1
Actual 2,0,3,13711


Accuracy Score : 0.9969849246231156
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      5383
           1       0.99      1.00      1.00     13738
           2       1.00      1.00      1.00     13714

    accuracy                           1.00     32835
   macro avg       1.00      0.99      1.00     32835
weighted avg       1.00      1.00      1.00     32835



In [35]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.6241103197597901, 'price'),
 (0.05230187401990237, 'house_size'),
 (0.04665600814928959, 'acre_lot'),
 (0.023012178441473678, 'bath'),
 (0.022633463012259117, 'remainder__price'),
 (0.021460622032453264, 'remainder__house_size'),
 (0.020312685388491222, 'remainder__acre_lot'),
 (0.01774098554419652, 'bed'),
 (0.015277638483633123, 'one_hot_encoder__state_Puerto Rico'),
 (0.009936760955130136, 'remainder__bed'),
 (0.0077920059503127775, 'one_hot_encoder__state_New Jersey'),
 (0.007688794867192518, 'remainder__bath'),
 (0.005485615945164884, 'one_hot_encoder__state_Massachusetts'),
 (0.0052981123956606195, 'one_hot_encoder__state_New York'),
 (0.004945646775917184, 'remainder__price_bin'),
 (0.003381068376763554, 'one_hot_encoder__state_Maine'),
 (0.002103101898368646, 'one_hot_encoder__state_Vermont'),
 (0.002055452884131049, 'one_hot_encoder__state_Rhode Island'),
 (0.0015797326623636302, 'one_hot_encoder__city_San Juan'),
 (0.0013119048358823774, 'one_hot_encoder__city_Bayamon'),
